In [ ]:
! pip install toon_python
!pip install datasets
import pandas as pd
from toon_python import encode
from datasets import load_dataset
import json
import ast

1. Importing dataset and conveting json dataset into toon format

In [ ]:
import re

In [ ]:
print("Downloading dataset...")
dataset = load_dataset("glaiveai/glaive-function-calling-v2", split="train[:50000]")

toon_data = []

print(f"Processing {len(dataset)} items...")

for row in dataset:
    text = row['chat']

    if "<functioncall>" in text:
        try:
            parts = text.split("<functioncall>")
            user_part_raw = parts[0]
            json_part_raw = parts[1]

            stop_markers = ["<|endoftext|>", "FUNCTION RESPONSE", "ASSISTANT"]
            for marker in stop_markers:
                if marker in json_part_raw:
                    json_part_raw = json_part_raw.split(marker)[0]

            json_str = json_part_raw.strip()

            outer = ast.literal_eval(json_str)
            outer["arguments"] = json.loads(outer["arguments"])

            toon = encode(outer)

            if "USER:" in user_part_raw:
                instruction = user_part_raw.split("USER:")[-1].replace("ASSISTANT:", "").strip()
            else:
                continue

            # append to full dataset
            toon_data.append({
                "instruction": instruction,
                "output": toon
            })
        except Exception:
            continue

if len(toon_data) > 0:
    df = pd.DataFrame(toon_data)
    df.to_csv("toon_training_data.csv", index=False)
    print(f"Success! Converted {len(df)} items using the official toon-format library.")
else:
    print("Still 0 items. Parsing logic might need adjustment.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/106 [00:00<?, ?B/s]

glaive-function-calling-v2.json:   0%|          | 0.00/271M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/112960 [00:00<?, ? examples/s]

Processing 50000 items...
Success! Converted 23761 items using the official toon-format library.
